Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

Load and visualize dataset

In [ ]:
#Load data
train_data = pd.read_csv('Train.csv')

#Split data and labels
X = train_data.iloc[:, 1:]  
y = train_data.iloc[:, 0]

#Process data
if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)  
X = X.values / 255.0
X = X.reshape(-1, 28, 28, 1)

#Visualize data as image on chosen index
plt.imshow(X[1].reshape(28, 28), cmap='gray')

SINGLE NEURON 

In [ ]:
def single_neuron(inputs):
    
    l = len(inputs)
    
    weights = np.random.rand(1,l)
    bias    = np.random.rand(1,1)
    out     = np.dot(weights,inputs) + bias
    return(out)

LAYER OF NEURONS

ACTIVATION FUNCTION 

SOFTMAX

LOSS AND ENTROPY 

BACKPROP 

OPTIMIZER

LEARNING AND TRAINING 

CONVOLUTION

POOLING

SIGMOID AND TANH

FLATTENING

BACKPROB AGAIN 

TESTING